Don't know how to make a sagemath package, so to use this library of functions, just type "%run ./Library.ipynb" at the top of whatever other Jupyter notebook you're trying to use these functions in.

In [5]:
import numpy as np
from sage.modules.misc import gram_schmidt

## Making vector / matrix things

In [6]:
def lib_uniform_rand_vec(dimension):
    """
    Makes a uniformly random vector.
    
    Args:
    - Desired dimension
    
    Returns:
    - Uniformly random vector of that dimension 
    """
    xs = np.random.normal(0,1,dimension) #normally random x's
    norm = sum(xs**2)**0.5 #and then normalize
    return vector(xs/norm)

#####################################################################################

def lib_flat_vec(dim):
    """
    Makes a flat vector.
    
    Args:
    - Desired dimension
    
    Returns:
    - Vector of that dimension with entries of modulus 1
    """
    theta = np.random.uniform(0,2*pi,dim)    
    vec = []
    for j in range(dim):
        vec.append(math.sin(theta[j])+math.cos(theta[j])*I)

    return vec

In [ ]:
def lib_Fourier_Matrix(d):
    """
    Makes the Fourier matrix associated with a specified dimension

    Args:
    - Dimension

    Returns:
    - Fourier matrix of that dimension
    """

    rs=[]
    for k in range(d):
        rw = [None for i in range(d)]
        for j in range(d):
            rw[j] = exp(2*math.pi*I*k*j/d)
        rs.append(rw)
    F = rs
    for k in range(d):
        for j in range(d):
            if abs(F[k][j].real()) < 1e-5:
                F[k][j]=F[k][j].imag()*I
            if abs(F[k][j].imag()) < 1e-5:
                F[k][j]=F[k][j].real()
    return (1/sqrt(d))*Matrix(F)


In [65]:
def lib_standard_basis_vectors(d):
    """
    Will give a list of the standard basis vectors in dimension d.
    
    Args:
    - Dimension d
    
    Returns:
    - List of standard basis vectors, each vector as a list
    """
    veclist = []
    for i in range(d):
        vec = []
        for j in range(d):
            if i == j:
                vec.append(1)
            else:
                vec.append(0)
        veclist.append(vec)
    return veclist

    

### Finding degeneracy of a graph and the vertex ordering that admits that degeneracy

In [21]:
def lib_degen_find_min(G1):
    """
    Finds the degeneracy of a given graph by plucking off minimum degree vertices,
    and placing them at the end of the optimal ordering
    
    Args:
    - The graph
    
    Returns:
    - Degeneracy of the graph
    - Ordering of vertices that admits that degeneracy
    """
    ### Finding the optimal ordering ###
    G=G1.copy()
    n = len(G.vertices());
    ordering = [None for i in range(n)];
    j = 0; #counts how many vertices we've deleted so far

    while j < n:
        degs = G.degree(i for i in G.vertices()) #degrees of all vertices in graph
        mind = min(degs); #mininum degree
        for k in G.vertices():
            if G.degree(k) == mind:
                ordering[n-1-j] = k; #if the vertex has min degree, put it at end of ordering
                G.delete_vertex(k); #and remove that vertex from the graph
                j = j+1
                break
    
    ### Finding the degeneracy of that ordering###
    npnv=[] #empty list of the number of previous neighbors
    
    for i in range(n):
        vi = ordering[i] #the vertex associated with index i in the optimal ordering
        npn=0 #number of previous neighbors to vertex vi
        for j in range(i): #for all previous vertices in the optimal ordering
            vj = ordering[j]
            if vj in G1.neighbors(vi): #if the vertex is adjacent to vertex vi, increase the number of previous neighbors of i
                npn+=1
        npnv.append(npn) #put the number of vi's previous neighbors into the list of number of previous neighbors
    degen = max(npnv) #the degeneracy is the max of all of those previous neighbors
    
    return ordering,degen

#####################################################################################
#####################################################################################

def lib_degen_find_max(G1):
    """
    Finds the degeneracy of a given graph by ordering the vertices from highest to lowest degree for the optimal ordering
    
    Args:
    - The graph
    
    Returns:
    - Degeneracy of the graph
    - Ordering of vertices that admits that degeneracy
    """
    
    ### Finding the optimal ordering ###
    G=G1.copy()
    n = len(G.vertices());
    ordering=[];
    j = 0; #counts how many vertices we've deleted so far
    degs = G.degree(i for i in G.vertices()) #degrees of all vertices in graph
    
    while j < n:
        maxd=max(degs)
        for k in G.vertices():
            if G1.degree(k) == maxd:
                ordering.append(k)
                degs.remove(maxd)
                G.delete_vertex(k)
                j=j+1
                break
            
    ### Finding the degeneracy of that ordering###
    npnv=[] #empty list of the number of previous neighbors
    for i in range(n):
        vi = ordering[i] #the vertex associated with index i in the optimal ordering
        npn=0 #number of previous neighbors to vertex vi
        for j in range(i): #for all previous vertices in the optimal ordering
            vj = ordering[j]
            if vj in G1.neighbors(vi): #if the vertex is adjacent to vertex vi, increase the number of previous neighbors of i
                npn+=1
        npnv.append(npn) #put the number of vi's previous neighbors into the list of number of previous neighbors
    degen = max(npnv) #the degeneracy is the max of all of those previous neighbors

    return ordering,degen

#####################################################################################
#####################################################################################

def lib_degen_find(G):
    """
    Finds minimum degeneracy by using both ways
    
    Args:
    - A graph
    
    Returns:
    - Minimum degeneracy of the graph using the two different methods
    - The vertex ordering that admits that degeneracy
    """
    return min(lib_degen_find_max(G)[1], lib_degen_find_min(G)[1])


## Modified LSS, finds orthogonal representation of graph using its degeneracy

In [22]:
def lib_modified_LSS(G,Vorder,d):
    """
    Finds an orthogonal representation of the graph
    
    Args:
    - The graph
    - An ordering of the vertices
    - Desired dimension of the orthogonal representation
    
    Returns:
    - An orthogonal representation of the graph
    """
    
    n = len(Vorder) #number of vertices
    us = [None for i in range(n)] #empty list in which we can put uniformly random vectors
    fs=[] #empty list into which we shall put orthogonal vectors

    for i in range(n): # assign uniformly random vectors to everything
        us[i] = vector(lib_uniform_rand_vec(d)) 
        
    fs.append(Matrix(us[0])); #first one is just the regular random guy
 
    for i in range(1,n):
        prev_f = []; #gather the previously assigned f vectors (the orthogonal ones)
        vi = Vorder[i]; #the vertex in the ith place
        
        nnp=0; #number of neighbors previous in the list
        for j in range(i): #for all the vertices previous in the listed order
            vj = Vorder[j]; #the vertex in the jth place
            
            if vj in G.neighbors(vi): #if the previous vertex is a neighbor of this one
                nnp+=1; #add one to the count of previous neighbors
                prev_f.append(vector(fs[j])) #collect its vector in the list of previous f vectors
                bassisor,b = Matrix(prev_f).gram_schmidt() #gram schmidtify the previous f vectors into an orthogonal basis
        
        newv = Matrix(us[i]) #the new vector will be the randomly generated u and then we'll mess with it
        if nnp != 0: #if there were neighbors previous in the list
    
            for v1 in bassisor: #for all vectors in that basis of prev fs
                v1vec=Matrix(v1); #make a matrix out of it
                newv -= Matrix((newv*v1vec.T)*v1vec) #subtract the projection of u onto that vec from u
            
        fs.append(newv) #normalize and add it to the list of f vectors
    return fs

## Making quantum colorings using the 4D and 8D construction methods

In [23]:
def lib_quat_tophats(vec):
    """
    Makes the four tophat vectors for quaternion construction from one 4D vector, and the associated projectors
    
    Args:
    - A 4D vector
    
    Returns:
    - A list of four orthogonal projectors
    """
    
    phi0 = Matrix(vec[0])
    phi1 = Matrix([-vec[0][1],vec[0][0],-vec[0][3],vec[0][2]])
    phi2 = Matrix([-vec[0][2],vec[0][3],vec[0][0],-vec[0][1]])
    phi3 = Matrix([-vec[0][3],-vec[0][2],vec[0][1],vec[0][0]])

    #now outerproduct dotify them to make projection matrices
    proj0 = phi0.T * phi0
    proj1 = phi1.T * phi1
    proj2 = phi2.T * phi2
    proj3 = phi3.T * phi3
    
    return proj0, proj1, proj2, proj3 #tadah
    
        
#####################################################################################
    
def lib_oct_tophats(vec):
    """
    Makes the eight tophat vectors for octonion construction from one 8D vector, and the associated projectors
    
    Args:
    - An 8D vector
    
    Returns:
    - A list of eight orthogonal projectors
    """
        
    tophat=[None for i in range(8)];
    tophat[0] = Matrix([ vec[0][0],  vec[0][1],  vec[0][2],  vec[0][3],  vec[0][4],  vec[0][5],  vec[0][6],  vec[0][7] ])
    tophat[1] = Matrix([-vec[0][1],  vec[0][0], -vec[0][3],  vec[0][2], -vec[0][5],  vec[0][4],  vec[0][7], -vec[0][6] ])
    tophat[2] = Matrix([-vec[0][2],  vec[0][3],  vec[0][0], -vec[0][1], -vec[0][6], -vec[0][7],  vec[0][4],  vec[0][5] ])
    tophat[3] = Matrix([-vec[0][3], -vec[0][2],  vec[0][1],  vec[0][0], -vec[0][7],  vec[0][6], -vec[0][5],  vec[0][4] ])
    tophat[4] = Matrix([-vec[0][4],  vec[0][5],  vec[0][6],  vec[0][7],  vec[0][0], -vec[0][1], -vec[0][2], -vec[0][3] ])
    tophat[5] = Matrix([-vec[0][5], -vec[0][4],  vec[0][7], -vec[0][6],  vec[0][1],  vec[0][0],  vec[0][3], -vec[0][2] ])
    tophat[6] = Matrix([-vec[0][6], -vec[0][7], -vec[0][4],  vec[0][5],  vec[0][2], -vec[0][3],  vec[0][0],  vec[0][1] ])
    tophat[7] = Matrix([-vec[0][7],  vec[0][6], -vec[0][5], -vec[0][4],  vec[0][3],  vec[0][2], -vec[0][1],  vec[0][0] ])
    
    cowboy=[];
    for i in range(8):
        cowboy.append(tophat[i].conjugate_transpose() * tophat[i])
    
    return cowboy

#####################################################################################
#####################################################################################
#####################################################################################
#####################################################################################

def lib_quat_const(vecs):
    """
    Constructs quantum coloring using quaternion construction
    
    Args:
    - List of 4D vectors in a real orthogonal representation
    
    Returns:
    - A quantum 4-coloring
    """
    
    n = len(vecs)
    qch = [];
    
    for j in range(n):
        qch.append(lib_quat_tophats(vecs[j]/vecs[j].norm()))
    return qch

#####################################################################################
def lib_oct_const(vecs):
    """
    Constructs quantum coloring using octonion construction
    
    Args:
    - List of 8D vectors in a real orthogonal representation
    
    Returns:
    - A quantum 8-coloring
    """
        
    n=len(vecs)
    qch = [];
    for j in range(n):
         qch.append(lib_oct_tophats(vecs[j]/vecs[j].norm()))
    return qch
    

In [24]:
def lib_reconstructing(vecs,ordering):
    """
    Reconstructs a graph from vector adjacencies
    
    Args:
    - List of vectors
    - Ordering of vertices
    
    Returns:
    - A graph with as many vertices as there are vectors, where vertices are adjacent if vectors are orthogonal
    """
    
    n = len(vecs) #how many vectors we have
    G_r = Graph() #empty graph
    G_r.add_vertices([i for i in range(n)]) #add vertices to the empty graph
    for i in range(n): #for all vertices
        for j in range(i+1,n):
            b=Matrix(vecs[i])*Matrix(vecs[j]).conjugate_transpose() #find the dot product of the vectors in the orthogonal representation
            if abs(b[0])<1e-9: #if that's about zero
                vi=ordering[i]; #name vi and vj as the ones in the order
                vj=ordering[j];
                G_r.add_edge([vi,vj]) #add an edge between vi and vj
    return G_r


In [25]:
def lib_degen_OR_recon(G): #puts previous guys together:
    """
    Finds degeneracy of graph and optimal vertex-ordering, constructs orthogonal representation, and reconstructs the graph
    
    Args:
    - The graph
    
    Returns:
    - A graph reconstructed from vector adjacencies
    """
    
    [ordering, degen] = lib_degen_find_max(G)
    vecs = lib_modified_LSS(G,ordering,degen+1)
    G_r = lib_reconstructing(vecs,ordering)
    
    return G_r


In [26]:
def lib_check_qch(qch,G,ordering):
    """
    Checks whether or not it's actually a quantum coloring
    
    Args:
    - Purported quantum coloring
    - The graph
    
    Returns:
    - IF FAILED, returns errors
    """
    
    n = len(qch);
    c = len(qch[0])
    d = len(qch[0][0][0])
    
    comp_errs = 0; #initialize number of completeness errors
    orth_errs = 0; #initialize number of orthogonality errors
    #checking completeness
    for vi in range(n):
        summ = sum(qch[vi]) #sum them all up
        for i in range(d):
            for j in range(d):
                if ((i == j) and (abs(summ[i,j]) >= 1+1e-5)) or ((i != j) and (abs(summ[i][j] > 1e-5))):
                    print('Failure on vertex ', G.vertices()[vi]) #if ON diagonal and NOT ~1, or OFF diagonal and NOT ~0, print where it failed
                    comp_errs +=1;

    #checking orthogonality
    #if they're different vertices and their respective qch matrices multiply to have trace not ~0, and they're adjacent
    #then print where they failed
    for i in range(n):
        vi = ordering[i]
        for j in range(i):
            vj = ordering[j];
            for ci in range(c):
                #if the vertices are different AND the trace of their product isn't zero AND they're neighbors, then it's failure
                #if ((vi != vj) and (abs((qch[vi][ci]*qch[vj][ci].conjugate_transpose()).trace()) > 1e-5) and (G.vertices()[vi] in G.neighbors(G.vertices()[vj]))):
                #if (  abs( (qch[i][ci]*qch[j][ci].conjugate_transpose() ).trace() ) > 1e-5  ) and ( vi in G.neighbors(vj)):
                vjMAT = qch[j][ci]
                viMAT = qch[i][ci]
                thetrace = (vjMAT*viMAT).trace()
                if (vi in G.neighbors(vj)) and (abs(thetrace) > 1e-5) :
                    print('Failure on vertices ', G.vertices()[vi], G.vertices()[vj],' on color ',ci)
                    print('With trace of ', thetrace)
                    #print('With trace of ',((qch[vi][ci]*qch[vj][ci].conjugate_transpose()).trace()))
                    #print('And matrices of ',((qch[vi][ci]*qch[vj][ci].conjugate_transpose())))
                    orth_errs +=1;
    
    if ((comp_errs == 0) and (orth_errs == 0)):
        print('All quiet on the errors front')
    else:
        print('Wow, whoops!\nCompleteness errors:', comp_errs,'\nOrthogonality errors: ',orth_errs)

    return


In [27]:
def lib_halve_qcoloring(qch): #bad function!
    """
    Cuts an 8D quantum 8-coloring down into being a 4D quantum 8-coloring

    Args:
    - An 8D quantum 8-coloring

    Returns:
    - A 4D quantum 8-coloring
    """
    
    n = len(qch);
    c = len(qch[0]);
    dhalf = len(qch[0][0][0])/2;
    hqch = [] #empty list for when we halve it
    
    for vi in range(n):
        listofmats = []; #empty list of halved matrices for this vertex
        for ci in range(c):
            #ignore the fact that I don't know how to use Sage and thus had to make a silly ass matrix
            #empt = Matrix([math.pi, math.pi, math.pi, math.pi]).T*Matrix([math.pi, math.pi, math.pi, math.pi])
            empt = Matrix(QQ,dhalf,dhalf,range(dhalf*dhalf))
            for i in range(dhalf):
                for j in range(dhalf):
                    empt[i,j] = qch[vi][ci][i][j]
            listofmats.append(empt)
        hqch.append(listofmats)

    return hqch
        

## Lower bounds on quantum chromatic number

In [281]:
def lib_spectral_lower_bounds_part1(G):
    """
    Takes the inequality from Spectral Lower Bounds Part 1 and applies it to a graph.

    Args:
    - A graph

    Returns:
    - Lower bound for quantum chromatic number of the graph
    """
    n = len(G.vertices())
    m = len(G.edges())
    
    A = G.adjacency_matrix()
    D = diagonal_matrix(vector(G.degree()))
    Q = D + A #signless Laplacian of G
    L = D - A #Laplacian of G
    
    mu1 = max(A.eigenvalues()); mun = min(A.eigenvalues()) #max and min eigenvalues of A
    delt1 = max(Q.eigenvalues()); deltn = min(Q.eigenvalues()) #max and min eigenvalues of Q
    thet1 = max(L.eigenvalues()); thetn = min(L.eigenvalues()) #max and min eigenvalues of L
    
    eigpos = [j for j in A.eigenvalues() if j > 0] #positive eig vals of A
    eigneg = [j for j in A.eigenvalues() if j < 0] #negative eig vals of A
    
    nplus = len(eigpos); nneg = len(eigneg) #number of positive and negative eigenvalues of A
    
    splus = sum([j^2 for j in eigpos]); sneg = sum([j^2 for j in eigneg]) #sum of squares of pos and neg eig vals of A
    
    ## VARIOUS BOUNDS ON CHROMATIC NUMBER IN EQN. (3) ##
    bound_1 = 1 + mu1 / abs(mun);                   #Hoffman [8]
    bound_2 = 1 + 2*m / (2* m - n*deltn);           #Lima et al[11]
    bound_3 = 1 + mu1 / (mu1 - delt1 + thet1);      #Kolotilina [10]
    bound_4 = 1 + max(nplus / nneg, nneg / nplus);  #Elphick and Wocjan [7]
    bound_5 = 1 + max(splus / sneg, sneg / splus);  #Ando and Lin [1]
    
    low_bound = max(bound_1, bound_2, bound_3, bound_4, bound_5); #Implementing Eqn. (3)
    
    ## TK TO DO: find out how to make the lovasz theta function part work. Section 6 of the paper.
    ### b1 \leq vector chromatic number (G) \leq lovasz theta function (G complement) \leq quantum chromatic number (G)
    return low_bound, bound_1, bound_2, bound_3, bound_4, bound_5

#####################################################################################
#####################################################################################

def lib_spectral_lower_bounds_part2(G):
    """
    Takes the inequality from Spectral Lower Bounds Part 2 and applies it to a graph.

    Args:
    - A graph

    Returns:
    - Lower bound for quantum chromatic number of the graph
    """
    
    A = G.adjacency_matrix()
    spec_A = A.eigenvalues() #eig vals of A
    spec_A_inc = spec_A.copy(); spec_A_inc.sort() #eig vals from lowest to highest
    spec_A_dec = spec_A.copy(); spec_A_dec.sort(reverse=1) #eig vals from highest to lowest
    
    spec_A_unique = list(set(spec_A)); spec_A_unique.sort(reverse = 1); #eigs of A w/o multiplicities from big to small
    
    umax = max(spec_A) #largest eigenvalue of A
    u2 = spec_A_unique[1] #second largest eigenvalue
    un = min(spec_A) #smallest eigenvalue
    
    ## FIRST BOUND ##
    summ = umax #implementing the inequality in Eqn. (20)
    for j in range(len(spec_A)):
        if summ <= 0:
            k=j
            break
        else:
            summ = summ + spec_A_inc[j]
        
    bound_1 = 1 + k #the lower bound in Eqn. (21)
    
    
    ## SECOND BOUND ##
    for (l,E) in A.right_eigenspaces(): #finding multiplicity of smallest eig val
        if l == un:
            g = E.dimension() #multiplicity of smallest eig val!
            break

    bound_2 = 1 + min(g,abs(un)/u2) #the lower bound in Eqn. (34)
    
    lower_bound = max(bound_1, bound_2)

    return lower_bound,bound_1, bound_2

#####################################################################################
#####################################################################################

def lib_all_lower_bounds(G):
    """
    Gives information on the lower bounds of the quantum chromatic number of the graph, as well as its chromatic number
    
    Args:
    - A graph
    
    Returns:
    - Print statement of chromatic number
    - Print statement of clique number
    - Print statement of spectral lower bounds part 1
    - Print statement of spectral lower bounds part 2
    """
    
    sp1 = [float(j) for j in lib_spectral_lower_bounds_part1(G)]
    sp2 = [float(j) for j in lib_spectral_lower_bounds_part2(G)]
    
    print('Chromatic number:\t\t',G.chromatic_number(),'\n')
    
    print('Lower bound:\t\t\t',max(G.clique_number(),max(sp1),max(sp2)),'\n')
    
    print('Clique number:\t\t\t',G.clique_number(),'\n')
    
    print('Spectral bounds 1:\t\t',ceil(sp1[0]))
    print('\tHoffman\t\t\t', sp1[1])
    print('\tLima\t\t\t', sp1[2])
    print('\tKolotilina\t\t', sp1[3])
    print('\tElphick and Wocjan\t', sp1[4])
    print('\tAndo and Lin\t\t', sp1[5])
    
    print('\nSpectral bounds 2:\t\t', ceil(sp2[0]))
    print('\tBound 1:\t\t',sp2[1])
    print('\tBound 2:\t\t',sp2[2])
    
    
    return

In [74]:
def lib_min_dim_OR(G):
    k = lib_degen_find(G)
    diam = (G.complement()).diameter()
    print('d >= k+1:\t d >=',k+1)
    print('d >= diam(Gc)+1: d >=',diam+1)
    return k+1,diam+1 #TK ????????
# lib_min_dim_OR(graphs.CirculantGraph(7,[1,2]))

## Making different types of graphs

In [73]:
def lib_group_graph(p,n,conn_set):
    """
    Will make a graph on Z_p^n with specified connection set.
    
    Args:
    - p for Z_p^n
    - n for Z_p^n
    - Connection set of vectors
    
    Returns:
    - A nice graph :-)
    """
    
    if len(set(list(map(len,conn_set)))) != 1: #check if vectors in connection set are valid
        raise ValueError("Vectors in the connection set must be of the same length")
    C = list(map(vector,conn_set)) #make the connection set into vectors
    
    S=Permutations([i for i in range(p)]*n,n).list() #permuatations of possibilities as a list
    veclist = [] #empty list
    
    for i in range(len(S)):
        veclist.append(vector(Integers(p),S[i])) #make a list of the vectorified possibilities
                       
    G = Graph() #empty graph
    
    for vec in veclist:
        for c in C:
            G.add_edge([tuple(vec),tuple(vec+c)]) #add edges between the vertices based on the connection set

    return G
    
    
#####################################################################################
#####################################################################################


def lib_make_Hadamard(k):
    """
    Makes Hadamard graph G_N
    
    Args:
    -k, for N = 4*k
    
    Returns:
    -Hadamard graph G_N
    """
    
    N = 4*k
    VG = Permutations([i for i in range(2)]*N,N).list() #vector names as a list
    G = Graph()
    
    veclist = [] #empty list
    for i in range(len(VG)):
        veclist.append(vector(VG[i])) #make a list of the vectorified possibilities

    for j in range(2^N):
        for k in range(j,2^N):
            if sum((veclist[j]+veclist[k])%2) == N/2: #if vectors have Hamming distance N/2, put an edge between them
                G.add_edge([tuple(veclist[j]),tuple(veclist[k])])
    return G

## Quantum protocol to win the graph coloring game on all Hadamard graphs.

In [282]:
def lib_avis_QFTN(k0):
    """
    Performs the quantum Fourier transform in dimension N, given in equation (1) of Avis et al.
    
    Args:
    -A tall skinny vector
    
    Returns:
    -The quantum Fourier transform of that vector
    """
    k1 = Matrix(k0) #if it's not a matrix, it is now
    if k1.ncols() < k1.nrows(): #if it's short fat, now it's tall skinny
        k = k1.transpose()
    else:
        k = k1
    N=k.nrows() #how long is the vector
    QFTN = lib_Fourier_Matrix(N)*sqrt(N) #make a Fourier matrix of that same dimension
    return QFTN*Matrix(CDF,k)#QFTN multiplies the Fourier matrix by the vector to do the thing

#####################################################################################
#####################################################################################


def lib_avis_phaseshift(L0):
    """
    Performs the phaseshift given in equation (2) of Avis et al.
    
    Args:
    -A tall skinny vector
    
    Returns:
    -The phaseshifted vector
    """
    L = Matrix(L0) #if it's not a matrix, it is now
    N = max(L.nrows(),L.ncols()) #length of vector
    PL=[]
    for j in range(N):
        PL.append((-1)^(L[j])*L[j])
    return PL


#####################################################################################
#####################################################################################


def lib_avis_CNOT(a0, b0):
    """
    Performs CNOT operations on target vector (b0) using control vector (a0), as given in equation (4) of Avis et al.
    
    Args:
    -Target vector b0
    -Control vector a0
    
    Returns:
    -CNOT'd version of the b0 vector
    """
    
    a1 = Matrix(a0) #if it's not a matrix, it is now
    b1 = Matrix(b0)
    
    if a1.nrows() < a1.ncols():a = a1.transpose() #if it's short fat, now it's tall skinny
    else: a = a1
    if b1.nrows() < b1.ncols():b = b1.transpose()
    else: b = b1
        
    N = a.nrows()
    CNOT_B = [None for i in range(N)];
    for j in range(N):
#         print(j)
        CNOT_B[j] = (RR(a[j][0])+RR(b[j][0]))
#     for j in range(N):
#         if a[j] == 0:
#             CNOT_B[j] = b[j]
#         elif a[j] == 1:
#             CNOT_B[j] = (b[j] + 1) %2
# #         else:
# #             print(a[j])
#     CNOT_B=a+b
#     CNOT_B = (a+b)%2 #adding the vectors together mod 2 will give the flipped b
    return CNOT_B